## Frameworks

In [1]:
import gurobipy as gp
from gurobipy import GRB
import json
import numpy as np
import pandas as pd
import time

## 1) Load Instance

In [2]:
start = time.time()

In [3]:
#path="instances/test.json"
path="instances/toy_instance.json"
#path="instances/medium_instance.json"
#path="instances/large_instance.json"

# JSON file
f = open (path, "r")
  
# Reading from file
data = json.loads(f.read())

In [4]:
# Gains
Gm=[]
for job in data["jobs"]:
    Gm.append(job['gain'])
Gm=np.array(Gm)

# Penalty
Pm=[]
for job in data["jobs"]:
    Pm.append(job['daily_penalty'])
Pm=np.array(Pm)

# Due Date
Dm=[]
for job in data["jobs"]:
    Dm.append(job['due_date'])
Dm=np.array(Dm)

# Penalty per day
Penalty=[]
for job in data["jobs"]:
    job_penalty=[]
    p=0
    for t in range(data["horizon"]):
        if (t>job["due_date"]-1):
            p+=job["daily_penalty"]
        job_penalty.append(p)
    Penalty.append(job_penalty)
Penalty=np.array(Penalty)

# Profit per day
Profit=[]
for job in data["jobs"]:
    job_benef=[]
    p=job['gain']
    for t in range(data["horizon"]):
        if (t>job["due_date"]-1):
            p-=job["daily_penalty"]
        job_benef.append(p)
    Profit.append(job_benef)
Profit=np.array(Profit)

# Vacations per day and per employee
Vacations=[]
for e in data["staff"]:
    e_planning=[]
    v=e['vacations']
    for t in range(data["horizon"]):
        if (t+1 in v):
            e_planning.append(1)
        else:
            e_planning.append(0)
    Vacations.append(e_planning)
Vacations=np.array(Vacations)

# Employee qualifications
Qualifications_staff=[]
for staff in data["staff"]:
    list_staff=[]
    for q in data["qualifications"]:
        if(q in staff["qualifications"]):
            list_staff.append(1)
        else:
            list_staff.append(0)
    Qualifications_staff.append(list_staff)
Qualifications_staff = np.array(Qualifications_staff)

# Jobs Qualifications
Qualifications_job=[]
for job in data["jobs"]:
    #working_days_per_qualification
    list_job=[]
    for q in data["qualifications"]:
        try:
            val=job['working_days_per_qualification'][q]
        except:
            val=0
        list_job.append(val)
    Qualifications_job.append(list_job)
Qualifications_job=np.array(Qualifications_job)

## 2) Model

In [5]:
###
# MODEL
model = gp.Model()

horizon=data["horizon"]
qualifs=len(data["qualifications"])
projects=len(data["jobs"])
staffs=len(data["staff"])

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-29


## 3) Variables

In [6]:
###
# VARIABLES

# Planning 
Planning = model.addMVar(
                        shape=(staffs,horizon,projects,qualifs),
                        vtype=GRB.BINARY,
                        name="Planning"
                        )

# Delay 
Delay_project = model.addVars(
                                projects,
                                lb=0,
                                ub=horizon,
                                vtype=GRB.INTEGER,
                                name="Delay_project"
                            ) 

# End Date 
End_project = model.addVars(
                                projects,
                                lb=1,
                                ub=horizon,
                                vtype=GRB.INTEGER,
                                name="End_project"
                            ) 

# Realisation Boolean
Realisation = model.addVars(
                                projects,
                                vtype=GRB.BINARY,
                                name="Realisation"
                            )

# Begin 

Begin_project = model.addVars(
                                projects,
                                lb=1,
                                ub=horizon,
                                vtype=GRB.INTEGER,
                                name="End_project"
                            ) 

## 4) Constraints

In [7]:
###
# CONSTRAINTS

#1 Uicity of affectation 
model.addConstrs(
    (gp.quicksum(gp.quicksum(Planning[staff][t]))<= 1
        for staff in range(len(data["staff"]))
        for t in range(data["horizon"])
    ),
    name="staff_qualif_unicity"
) 

#2 Vacations
model.addConstrs(
    (Planning[staff][t][job][qualif]<=1-Vacations[staff][t] 
        for staff in range(len(data["staff"]))
        for t in range(data["horizon"])
        for job in range(len(data["jobs"]))
        for qualif in range(len(data["qualifications"]))
    ),
    name="Vacations_constraint"
) 


#3 Qualification staff
model.addConstrs(
    (Planning[staff][t][job][qualif]<=Qualifications_staff[staff][qualif]
        for staff in range(len(data["staff"]))
        for t in range(data["horizon"])
        for job in range(len(data["jobs"]))
        for qualif in range(len(data["qualifications"]))
    ),
    name="Qualif_staff_constraint"
) 


# Definition d'une liste de la somme des temps de qualifs par projet
Max_temp_qualif_job=[]
for job in range(len(data["jobs"])):
    l_job=[]
    for qualif in range(len(data["qualifications"])):
        temp_qualif_job=0
        for t in range(data["horizon"]):
            for staff in range(len(data["staff"])):
                temp_qualif_job+=Planning[staff][t][job][qualif]
        l_job.append(temp_qualif_job)
    Max_temp_qualif_job.append(l_job)
Max_temp_qualif_job


#4 Couverture des qualifs => realisation
model.addConstrs(
        (Max_temp_qualif_job[job][qualif] >= Realisation[job] *  Qualifications_job[job][qualif]
            for job in range(len(data["jobs"]))
            for qualif in range(len(data["qualifications"]))
        ), name = "cover_qualif"
)

#5 Unicity realisation
model.addConstrs(
    (Max_temp_qualif_job[job][qualif] <= Qualifications_job[job][qualif]
            for job in range(len(data["jobs"]))
            for qualif in range(len(data["qualifications"]))
), name = "project_done_once")

#6 end project 
model.addConstrs(
   (Planning[staff][t][job][qualif] * t <= End_project[job]
        for staff in range(len(data["staff"]))
        for t in range(data["horizon"])
        for job in range(len(data["jobs"]))
        for qualif in range(len(data["qualifications"]))
), name = "end_project_date")

#7 Delay 
model.addConstrs(
    (End_project[job] - Dm[job] <= Delay_project[job]
       for job in range(len(data["jobs"]))
    ), name="delay_project"
)

#9 Begin Date to compute Z3
print("#9 Begin Date to compute Z3")


#8 Don't begin a project that is not done
print("#8 Don't begin a project that is not done")



Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
#9 Begin Date to compute Z3
#8 Don't begin a project that is not done


In [8]:
"""for project in range(data["jobs"]):
    for t in range(horizon):
        for staff in range(staffs):
            Planning[staff][t][job]Begin_project"""

'for project in range(data["jobs"]):\n    for t in range(horizon):\n        for staff in range(staffs):\n            Planning[staff][t][job]Begin_project'

## 5) Objectives

In [9]:
model.ModelSense = GRB.MAXIMIZE

### Z1: Maximize Profit

In [10]:
#Z1 = gp.quicksum(Realisation.values()*(Gm-Pm*Delay_project.values()))
Z1 = gp.quicksum((Realisation.values()*Gm)-(Pm*Delay_project.values()))
"""
model.setObjective(
        Z1,
        GRB.MAXIMIZE,
    )"""

'\nmodel.setObjective(\n        Z1,\n        GRB.MAXIMIZE,\n    )'

In [11]:
model.setObjectiveN(Z1, index=0, priority=3, name='Jobs', weight=1)

"""model.setObjectiveN(
        expr=Z1,
        index=0,
        priority=1,
        weight=1,
        name="profit"
    )"""

'model.setObjectiveN(\n        expr=Z1,\n        index=0,\n        priority=1,\n        weight=1,\n        name="profit"\n    )'

### Z2: Minimize Number of Projects per Employee

In [12]:
Include_Z2 = True

In [13]:
# Jobs done by one staff
jobs_staffeur = model.addMVar(shape=(staffs,projects),vtype=GRB.BINARY,name="jobs_staffeur")

for staff in range(staffs):
    for job in range(projects):
        val=[]
        for t in range(horizon):
            for qualif in range(qualifs):
                val.append(Planning[staff][t][job][qualif])
        #real_jobs_staffeur[staff][job]=max(val)
        model.addConstr((jobs_staffeur[staff][job] == gp.max_(val)))

# Sum of all jobs for one staff
sum_staff_jobs = model.addVars(staffs,vtype=GRB.INTEGER, name="sum_staff_jobs")         

for staff in range(staffs):
    model.addConstr((sum_staff_jobs[staff]==gp.quicksum(jobs_staffeur[staff])))


In [14]:
if Include_Z2:
    for staff in range(staffs):
    #Z2 = gp.quicksum(sum_staff_jobs.values())
        model.setObjectiveN(
                    sum_staff_jobs[staff],
                    GRB.MINIMIZE,
                    1,# priority
                    0 # weight
                )

In [15]:
if Include_Z2:
    for staff in range(staffs):
        model.setObjectiveN(sum_staff_jobs[staff], 
                            index=1+staff, 
                            priority=2, 
                            weight=-1)  #weight = -1 to minimize


### Z3: Minimize Realisation Time

In [16]:
Include_Z3 = False

In [17]:
"""staff_list=[]
for staff in range(len(data["staff"])):
    job_list=[]
    for job in range(len(data["jobs"])):
        job_val=0
        for t in range(data["horizon"]):
            for qualif in range(len(data["qualifications"])):
                job_val+=Planning[staff][t][job][qualif]
        if(job_val>gp.LinExpr(0)):
            job_list.append(1)
        else:
            job_list.append(0)
    staff_list.append(job_list)

sum_per_employee=[]
for staff in range(len(data["staff"])):
    sum_per_employee.append(sum(normalized_staff_list[staff]))


# Create a variable for the objective
obj = model.addVar(lb=0, name="Total Assigned Jobs")

# Define the objective
model.setObjective(obj, GRB.MINIMIZE)

# Define the constraint
model.addConstr(obj >= sum(sum_per_employee), "Ensure the total number of assigned jobs is equal to or greater than the sum of jobs per employee")
"""

'staff_list=[]\nfor staff in range(len(data["staff"])):\n    job_list=[]\n    for job in range(len(data["jobs"])):\n        job_val=0\n        for t in range(data["horizon"]):\n            for qualif in range(len(data["qualifications"])):\n                job_val+=Planning[staff][t][job][qualif]\n        if(job_val>gp.LinExpr(0)):\n            job_list.append(1)\n        else:\n            job_list.append(0)\n    staff_list.append(job_list)\n\nsum_per_employee=[]\nfor staff in range(len(data["staff"])):\n    sum_per_employee.append(sum(normalized_staff_list[staff]))\n\n\n# Create a variable for the objective\nobj = model.addVar(lb=0, name="Total Assigned Jobs")\n\n# Define the objective\nmodel.setObjective(obj, GRB.MINIMIZE)\n\n# Define the constraint\nmodel.addConstr(obj >= sum(sum_per_employee), "Ensure the total number of assigned jobs is equal to or greater than the sum of jobs per employee")\n'

In [18]:
days_per_project = model.addVars(projects,vtype=GRB.INTEGER, name="days_per_project")

# constraint for the number of days needed for each project
for job in range(projects):
    s = 0
    for staff in range(staffs):
        for t in range(horizon):
            for qualif in range(qualifs):
                s += Planning[staff,t,job,qualif]
    model.addConstr(days_per_project[job] == s)


In [19]:
if Include_Z3:
    Z3 = gp.quicksum(days_per_project.values())

    model.setObjective(
                Z3,
                GRB.MINIMIZE
            )

## 6) Optimize

In [21]:
# epsilon

In [22]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 733 rows, 268 columns and 1798 nonzeros
Model fingerprint: 0x04f08c40
Model has 15 general constraints
Variable types: 0 continuous, 268 integer (245 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 5e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 609 rows and 144 columns
Presolve time: 0.00s
Presolved: 124 rows and 124 columns
---------------

## 7) Results

In [23]:
end = time.time()
print("Time taken: ", round(end - start,2), "seconds")
print("Time taken: ", round((end - start)/60,2), "minutes")
# toy : 1s
# medium : 53s
#large : 3,7 min

Time taken:  1.17 seconds
Time taken:  0.02 minutes


### Objectives

In [24]:
# Z1
print("Best Profit : ",model.objVal,"\nSum possible gains : ",Gm.sum())

Best Profit :  70.0 
Sum possible gains :  80


In [25]:
# Z2
for i in range(len(data["staff"])):
    print(sum_staff_jobs[i])

<gurobi.Var sum_staff_jobs[0] (value 2.0)>
<gurobi.Var sum_staff_jobs[1] (value 2.0)>
<gurobi.Var sum_staff_jobs[2] (value 3.0)>


In [26]:
# Z3
for i in range(len(data["jobs"])):
    print(days_per_project[i])

<gurobi.Var days_per_project[0] (value 3.0)>
<gurobi.Var days_per_project[1] (value 3.0)>
<gurobi.Var days_per_project[2] (value 3.0)>
<gurobi.Var days_per_project[3] (value 3.0)>
<gurobi.Var days_per_project[4] (value 0.0)>


In [37]:
begin=np.zeros(projects)
for job in range(projects):
    val=0
    for t in range(horizon):
        if(sum(sum(Planning[staff][t].x))==0):
                    val+=1      
    begin[job]=val
begin

array([1., 1., 1., 1., 1.])

### Constraints

In [27]:
for i in range(len(data["jobs"])):
    print(Realisation[i])

<gurobi.Var Realisation[0] (value 1.0)>
<gurobi.Var Realisation[1] (value 1.0)>
<gurobi.Var Realisation[2] (value 1.0)>
<gurobi.Var Realisation[3] (value 1.0)>
<gurobi.Var Realisation[4] (value 0.0)>


In [28]:
for i in range(len(data["jobs"])):
    print(Delay_project[i])

<gurobi.Var Delay_project[0] (value 0.0)>
<gurobi.Var Delay_project[1] (value 0.0)>
<gurobi.Var Delay_project[2] (value 0.0)>
<gurobi.Var Delay_project[3] (value 0.0)>
<gurobi.Var Delay_project[4] (value -0.0)>


### Visualization

In [29]:
# 4D matrix, Gurobi variables extraction
matrix = np.array([[element.x for element in row] for row in Planning]).reshape(staffs, horizon, projects, qualifs)

# 4D matrix to dataframe
df = pd.DataFrame(np.reshape(matrix, (staffs * horizon, projects * qualifs)),
                  columns=[f"Project {i+1} - Qualification {j+1}" for i in range(projects) for j in range(qualifs)],
                  index=[f"Staff {i+1} - Horizon {j+1}" for i in range(staffs) for j in range(horizon)])

# Add a row of sums
df.loc['sum_row'] = df.sum()

# Add a column of sums
df['sum_column'] = df.sum(axis=1)

df.to_csv("Results/planning_"+path.split("/")[-1].split('.')[0]+".csv",sep=";")

In [30]:
def highlight_cells(val):
    color = 'yellow' if val == 1 else 'red' if val > 1 else ''
    return 'background-color: %s' % color

df.style.applymap(highlight_cells)

,Project 1 - Qualification 1,Project 1 - Qualification 2,Project 1 - Qualification 3,Project 2 - Qualification 1,Project 2 - Qualification 2,Project 2 - Qualification 3,Project 3 - Qualification 1,Project 3 - Qualification 2,Project 3 - Qualification 3,Project 4 - Qualification 1,Project 4 - Qualification 2,Project 4 - Qualification 3,Project 5 - Qualification 1,Project 5 - Qualification 2,Project 5 - Qualification 3,sum_column
Staff 1 - Horizon 1,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
Staff 1 - Horizon 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
Staff 1 - Horizon 3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
Staff 1 - Horizon 4,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
Staff 1 - Horizon 5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Staff 2 - Horizon 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Staff 2 - Horizon 2,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
Staff 2 - Horizon 3,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
Staff 2 - Horizon 4,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
Staff 2 - Horizon 5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
